In [51]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [52]:
# Import data set
df = pd.read_csv('../data/artDataset_preprocessed.csv')

# Let's transform the data set into a numpy array
data_array = df.to_numpy()

# Predictors
X = data_array[:,1:]

# Target
y = data_array[:,0]

# **1. Baseline Model: Mean Predictor**

Let's apply a linear regression model with no features, i.e. it computes the mean of y on the training data, and use this value to predict y on the test data

In [58]:
# Implement a baseline model (mean predictor)
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import KFold, cross_val_score

# Set up K-Fold Cross-Validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=14)

# Initialize the baseline model (predicts the mean of y on the training set and uses it to predict y on the test set)
baseline_model = DummyRegressor(strategy='mean')

# Perform cross-validation on the baseline model
baseline_cv_scores_r2 = cross_val_score(baseline_model, X, y, cv=kf, scoring='r2')
baseline_cv_scores_neg_mse = cross_val_score(baseline_model, X, y, cv=kf, scoring='neg_mean_squared_error')

# Convert negative MSE to positive MSE and then to RMSE
baseline_cv_scores_mse = -baseline_cv_scores_neg_mse
baseline_cv_scores_rmse = np.sqrt(baseline_cv_scores_mse)

# Print baseline cross-validation results
print("\nBaseline Model (Mean Predictor) Cross-Validation Results:")
print(f"R² scores for each fold: {baseline_cv_scores_r2}")
print(f"Mean R²: {baseline_cv_scores_r2.mean():.4f} (±{baseline_cv_scores_r2.std():.4f})")
print(f"RMSE for each fold: {baseline_cv_scores_rmse}")
print(f"Mean RMSE: {baseline_cv_scores_rmse.mean():.4f} (±{baseline_cv_scores_rmse.std():.4f})")



Baseline Model (Mean Predictor) Cross-Validation Results:
R² scores for each fold: [-0.0032902  -0.00793626 -0.00072193 -0.03247125 -0.11076868]
Mean R²: -0.0310 (±0.0414)
RMSE for each fold: [11125.28975422 14912.55235547 11240.15193215 17734.83514548
  6841.09045667]
Mean RMSE: 12370.7839 (±3705.1512)


We observe that this models r2 is lightly below zero, indicating the mean‐only predictor performs worse than using the sample mean of the full dataset

## **2. Baseline Model: OLS**

Both the scaled and non-scaled models should have the same (similar) performance.

### Non-scaled

In [59]:
# Implement OLS baseline model
from sklearn.linear_model import LinearRegression
import numpy as np

# Initialize the OLS model
ols_model = LinearRegression()

# Use the same K-Fold Cross-Validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=14)

# Perform cross-validation on the OLS model
ols_cv_scores_r2 = cross_val_score(ols_model, X, y, cv=kf, scoring='r2')
ols_cv_scores_neg_mse = cross_val_score(ols_model, X, y, cv=kf, scoring='neg_mean_squared_error')

# Convert negative MSE to positive MSE and then to RMSE
ols_cv_scores_mse = -ols_cv_scores_neg_mse
ols_cv_scores_rmse = np.sqrt(ols_cv_scores_mse)

# Print OLS cross-validation results
print("\nOLS Model Cross-Validation Results:")
print(f"R² scores for each fold: {ols_cv_scores_r2}")
print(f"Mean R²: {ols_cv_scores_r2.mean():.4f} (±{ols_cv_scores_r2.std():.4f})")
print(f"RMSE for each fold: {ols_cv_scores_rmse}")
print(f"Mean RMSE: {ols_cv_scores_rmse.mean():.4f} (±{ols_cv_scores_rmse.std():.4f})")



OLS Model Cross-Validation Results:
R² scores for each fold: [ 0.05018181  0.11863282  0.0973063   0.06897914 -0.27193074]
Mean R²: 0.0126 (±0.1442)
RMSE for each fold: [10824.76026325 13944.85123917 10675.43743215 16841.00009194
  7320.5759778 ]
Mean RMSE: 11921.3250 (±3231.7006)


### Scaled

At first, I obtained wildly different (even astronomically large) R² and RMSE when I threw scaling into the mix. In theory, an ordinary least‐squares fit is invariant to affine rescaling of the features: if we scale every column by a constant and then un‐scale the coefficients, predictions end up exactly the same, so the R²/MAE/RMSE shouldn’t change.

When we get huge negatives like –1e14 for R² or RMSE on the order of 1e11, it almost always means that your pipeline has become numerically unstable.

Solution: get rid of the features with null variance or establish a threshold

In [ ]:
# 5-fold setup
kf = KFold(n_splits=5, shuffle=True, random_state=14)

from sklearn.feature_selection import VarianceThreshold

pipeline = Pipeline([
    ("var_thresh", VarianceThreshold()),         # removes features with σ=0
    ("scaler",    StandardScaler()),
    ("ols",       LinearRegression())
])

# CV on R² and RMSE
r2_scores      = cross_val_score(pipeline, X, y, cv=kf, scoring="r2")
neg_mse_scores = cross_val_score(pipeline, X, y, cv=kf, scoring="neg_mean_squared_error")
rmse_scores    = np.sqrt(-neg_mse_scores)

print("R² per fold:      ", r2_scores)
print("Mean R²:           ", f"{r2_scores.mean():.4f} ± {r2_scores.std():.4f}")
print("RMSE per fold:     ", rmse_scores)
print("Mean RMSE:         ", f"{rmse_scores.mean():.2f} ± {rmse_scores.std():.2f}")

Best number of neighbors: 9
Best R² score:            0.0866
RMSE per fold:            [ 9691.57077991 13077.34177415 12046.57615531 16330.09420118
  6509.66310275]
Mean RMSE:                11531.05 ± 3295.02


In [72]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

# 5-fold CV setup
kf = KFold(n_splits=5, shuffle=True, random_state=14)

# Define pipeline
pipeline = Pipeline([
    ("var_thresh", VarianceThreshold()),
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])

# Define the parameter grid for 'n_neighbors'
param_grid = {
    "knn__n_neighbors": [1, 3, 5, 7, 9, 11]
}

# Setup GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring="r2", return_train_score=True)
grid_search.fit(X, y)

# Best model and its parameters
print("Best number of neighbors:", grid_search.best_params_["knn__n_neighbors"])
print("Best R² score:           ", f"{grid_search.best_score_:.4f}")

# Optional: Compute RMSE with best estimator
best_model = grid_search.best_estimator_
neg_mse_scores = cross_val_score(best_model, X, y, cv=kf, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-neg_mse_scores)

print("RMSE per fold:           ", rmse_scores)
print("Mean RMSE:               ", f"{rmse_scores.mean():.2f} ± {rmse_scores.std():.2f}")


Best number of neighbors: 9
Best R² score:            0.0866
RMSE per fold:            [ 9691.57077991 13077.34177415 12046.57615531 16330.09420118
  6509.66310275]
Mean RMSE:                11531.05 ± 3295.02
